In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
import math

print("Tensorflow version " + tf.__version__)

In [ ]:
try:
    # 「TPU VM」では、このシンプルなコードでTPUを検出します
    strategy = tf.distribute.TPUStrategy()
    print('TPU found!')
except Exception as e:
    print(e)
    # TPUが見つからない場合は、デフォルトの戦略を使用
    print('Falling back to default strategy')
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# GCS (Google Cloud Storage)上のデータセットへのパスを取得
# TPUはGCSから直接データを読み込むことで高速化される
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# --- パラメータ設定 ---
IMAGE_SIZE = [512, 512]
EPOCHS = 20
# バッチサイズはTPUのコア数(8)の倍数にすると効率が良い
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

# データセットのファイルパスを取得
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-512x512'
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

# クラス名 (104種類の花)
CLASSES = ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'wild geranium', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'ball moss', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barberton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'daisy', 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura', 'geranium', 'orange dahlia', 'pink-yellow dahlia', 'cautleya spicata', 'japanese anemone', 'black-eyed susan', 'silverbush', 'californian poppy', 'osteospermum', 'spring crocus', 'bearded iris', 'windflower', 'tree poppy', 'gazania', 'azalea', 'water lily', 'rose', 'thorn apple', 'morning glory', 'passion flower', 'lotus', 'toad lily', 'anthurium', 'frangipani', 'hibiscus', 'balloon flower', 'foxglove', 'bougainvillea', 'camellia', 'mallow', 'mexican petunia', 'bromelia', 'blanket flower', 'trumpet creeper', 'blackberry lily', 'common tulip', 'wild rose', 'watercress', 'magnolia', 'cyclamen ', 'tree mallow', 'english marigold', 'butterbur', 'columbine', 'desert-rose', 'tree of heaven', 'standing cypress', 'gladiolus']

In [ ]:
def decode_image(image_data):
    """TFRecordから読み込んだバイナリデータを画像形式にデコード"""
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # [0, 255] -> [0, 1]に正規化
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_labeled_tfrecord(example):
    """ラベル付きデータ（学習/検証用）をパースする"""
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label

def read_unlabeled_tfrecord(example):
    """ラベルなしデータ（テスト用）をパースする"""
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum

def data_augment(image, label):
    """データ拡張（学習データにのみ適用）"""
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, 0.9, 1.1)
    image = tf.image.random_brightness(image, 0.1)
    # 他にも回転、ズーム、カットアウトなど様々な手法がある
    return image, label

def get_dataset(filenames, labeled=True, ordered=False, augmented=False):
    """TFRecordファイルからtf.data.Datasetを構築する"""
    AUTO = tf.data.experimental.AUTOTUNE
    
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    
    # 順序を保つ必要がない場合は、シャッフルや並列処理を効率化
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = dataset.with_options(ignore_order)
    
    # パース処理
    if labeled:
        dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    else:
        dataset = dataset.map(read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    
    # データ拡張
    if augmented:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
        
    dataset = dataset.repeat() # 学習中にデータが尽きないようにリピート
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=AUTO) # 次のバッチをバックグラウンドで準備
    return dataset

# データセットのインスタンスを作成
train_dataset = get_dataset(TRAINING_FILENAMES, labeled=True, augmented=True)
valid_dataset = get_dataset(VALIDATION_FILENAMES, labeled=True)
test_dataset = get_dataset(TEST_FILENAMES, labeled=False, ordered=True)

In [ ]:
with strategy.scope():
    # ImageNetで事前学習済みのEfficientNetB7を読み込む
    # `include_top=False`で最終層（1000クラス分類層）を除外
    pretrained_model = tf.keras.applications.EfficientNetB7(
        weights='imagenet',
        include_top=False,
        input_shape=[*IMAGE_SIZE, 3]
    )
    pretrained_model.trainable = True # 転移学習のためにモデル全体を再学習可能にする

    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(), # 特徴マップをベクトル化
        tf.keras.layers.Dense(len(CLASSES), activation='softmax') # 104クラス分類の出力層
    ])

# モデルのコンパイル
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

model.summary()

In [ ]:
# データセットに含まれる画像の総数を計算
num_train_images = int(12753) # 事前に数えておく
num_valid_images = int(3712)
num_test_images = int(7382)
steps_per_epoch = num_train_images // BATCH_SIZE
validation_steps = num_valid_images // BATCH_SIZE

print("学習を開始します...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset,
    validation_steps=validation_steps
)

In [ ]:
print("テストデータで予測を行います...")
# テストデータセットはリピートしないように再設定
test_ds_for_predict = get_dataset(TEST_FILENAMES, labeled=False, ordered=True)
test_images_ds = test_ds_for_predict.map(lambda image, idnum: image)
test_ids_ds = test_ds_for_predict.map(lambda image, idnum: idnum).unbatch()

# 予測の実行
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)

# IDを取得
test_ids = next(iter(test_ids_ds.batch(num_test_images))).numpy().astype('U')

# 提出用DataFrameを作成
submission = pd.DataFrame(data={'id': test_ids, 'label': predictions})
submission.to_csv('submission.csv', index=False)

print("提出ファイル 'submission.csv' を作成しました。")
print(submission.head())